# Ferramenta de Análise Astronômica (Jupyter Notebook)

Bem-vindo à versão Jupyter Notebook da **Ferramenta de Análise Astronômica**!

Este notebook permite que você planeje observações astronômicas de forma interativa, personalizável e integrada com seus próprios scripts Python.

---

## 🚀 Como Usar Este Notebook

1. **Configure suas preferências** na seção 2
2. **Execute as células sequencialmente** (Shift+Enter)
3. **Analise os resultados visuais** gerados automaticamente

---

## 📚 1. Importação de Módulos

In [ ]:
import warnings
from datetime import date
import pytz, csv, os
from src.config import *
from src.location import *
from src.targets import *
from src.analysis import *
from src.plotting import *
warnings.filterwarnings('ignore', category=AstropyWarning)
print("Módulos carregados com sucesso!")

---

## ⚙️ 2. Configurações da Análise

Edite as variáveis abaixo conforme suas necessidades:

In [ ]:
from datetime import date

# Localização do observador
NOME_DA_CIDADE = "Vitória da Conquista, Brazil"

# Data para análise noturna
DATA_ANALISE = date(2024, 7, 15)

# Elevação mínima do alvo acima do horizonte (em graus)
ELEVACAO_MINIMA_GRAUS = 30

# Seleção de alvos
usar_alvos_predefinidos = True
usar_alvos_sistema_solar = True
alvos_manuais = ["NGC 5128", "M83"]

# Arquivo de alvos (opcional)
ARQUIVO_DE_ALVOS = None  # Ou "targets.csv" se tiver um arquivo

# Configurações para análise anual
ALVO_ANUAL = "M42"
ANO_ANALISE = 2024

print("✅ Configurações definidas!")

---

## 🌙 3. Execução da Análise Noturna

Esta seção analisa a visibilidade de múltiplos alvos durante a noite selecionada.

In [ ]:
# Obter localização do observador
observer_location = get_location_from_city(NOME_DA_CIDADE)
observer_timezone = set_timezone_for_sao_paulo(observer_location) or pytz.UTC

# Inicializar variáveis SEMPRE (mesmo se houver erro)
all_targets = {}
night_events = {}

if observer_location is not None:
    # Calcular eventos noturnos
    night_events = calculate_nightly_events(DATA_ANALISE, observer_location, observer_timezone)
    start_night, end_night = night_events['inicio_noite'], night_events['fim_noite']
    
    print(f"📅 Análise para: {DATA_ANALISE}")
    print(f"🌅 Pôr do Sol: {night_events['por_do_sol'].to_datetime():%H:%M UTC}")
    print(f"🌌 Início da Noite: {start_night.to_datetime():%H:%M UTC}")
    print(f"🌄 Fim da Noite: {end_night.to_datetime():%H:%M UTC}")
    print()
    
    # Coletar nomes de alvos
    nomes_alvos = []
    if usar_alvos_predefinidos:
        nomes_alvos.extend(DEEP_SKY_TARGETS_PRESET)
    if alvos_manuais:
        nomes_alvos.extend(alvos_manuais)
    if ARQUIVO_DE_ALVOS and os.path.exists(ARQUIVO_DE_ALVOS):
        if ARQUIVO_DE_ALVOS.endswith('.txt'):
            with open(ARQUIVO_DE_ALVOS, 'r') as f:
                nomes_alvos.extend([l.strip() for l in f if l.strip() and not l.startswith('#')])
        elif ARQUIVO_DE_ALVOS.endswith('.csv'):
            with open(ARQUIVO_DE_ALVOS, 'r') as f:
                reader = csv.DictReader(f)
                nomes_alvos.extend([row['alvo'] for row in reader if 'alvo' in row])
    
    # Buscar coordenadas
    if nomes_alvos:
        all_targets.update(get_target_skycoords(nomes_alvos))
    if usar_alvos_sistema_solar:
        all_targets.update(registrar_alvos_sistema_solar(start_night))
    
    # Analisar e plotar cada alvo
    print(f"\n🔭 Analisando {len(all_targets)} alvos...\n")
    for name, coord in all_targets.items():
        if coord is not None:
            df = analyze_target_visibility_for_night(
                start_night, end_night, observer_location, coord, ELEVACAO_MINIMA_GRAUS * u.deg
            )
            
            if not df.empty:
                print(f"✅ {name}: Visível!")
                fig = plot_target_visibility(df, name, DATA_ANALISE, ELEVACAO_MINIMA_GRAUS)
                plt.show()
            else:
                print(f"❌ {name}: Não visível acima de {ELEVACAO_MINIMA_GRAUS}°")
else:
    print("❌ Não foi possível obter a localização. Verifique o nome da cidade.")

---

## 🗺️ 3.5. Mapa do Céu Noturno

### Versões Disponíveis:

**Célula seguinte (RECOMENDADA):** Teste simples e direto
- Edite o valor `HORA_ESCOLHIDA` e execute
- Sem widgets, sem complicações
- Funciona sempre!

**Próxima célula:** Interface com botão (experimental)
- Pode não funcionar dependendo do ambiente Jupyter

In [ ]:
# TESTE SIMPLES - Execute esta célula para gerar o mapa diretamente
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

print("🧪 Teste Direto do Mapa do Céu")
print("="*50)

# Verificar se as variáveis existem
try:
    print(f"✅ observer_location: {observer_location}")
    print(f"✅ all_targets: {len(all_targets)} alvos")
    print(f"✅ DATA_ANALISE: {DATA_ANALISE}")
    print(f"✅ ELEVACAO_MINIMA_GRAUS: {ELEVACAO_MINIMA_GRAUS}°")
except NameError as e:
    print(f"❌ Erro: {e}")
    print("Execute a célula de Análise Noturna primeiro!")
    raise

# Escolher horário (você pode mudar este valor)
HORA_ESCOLHIDA = 23  # Altere para a hora desejada (0-23)

print(f"\n🕐 Gerando mapa para {HORA_ESCOLHIDA:02d}:00 UTC...\n")

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if HORA_ESCOLHIDA >= 12:
    map_dt = base_dt.replace(hour=HORA_ESCOLHIDA, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=HORA_ESCOLHIDA, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)

for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f"🗺️ {len(visible)} alvos visíveis: {list(visible.keys())}\n")
    
    # Fechar figuras anteriores
    plt.close('all')
    
    # Gerar e mostrar
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
    
    print("\n✅ Mapa gerado!")
else:
    print(f"❌ Nenhum alvo visível às {HORA_ESCOLHIDA:02d}:00 UTC")
    print(f"   Alvos disponíveis: {list(all_targets.keys())}")
    print(f"   Elevação mínima: {ELEVACAO_MINIMA_GRAUS}°")

---

### 🕐 Mapas para Horas Comuns de Observação

Execute qualquer célula abaixo para ver o céu naquele horário:
- Cada célula já está configurada para um horário específico
- Apenas execute com Shift+Enter
- Sem necessidade de editar código!

In [ ]:
# Início da Noite (20:00 UTC)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

hora_escolhida = 20

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if hora_escolhida >= 12:
    map_dt = base_dt.replace(hour=hora_escolhida, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=hora_escolhida, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)
for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f'🗺️ Mapa do Céu - 20:00 UTC')
    print(f'✅ {len(visible)} alvos visíveis\n')
    
    plt.close('all')
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
else:
    print(f'❌ Nenhum alvo visível às 20:00 UTC')

In [ ]:
# Noite (22:00 UTC)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

hora_escolhida = 22

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if hora_escolhida >= 12:
    map_dt = base_dt.replace(hour=hora_escolhida, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=hora_escolhida, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)
for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f'🗺️ Mapa do Céu - 22:00 UTC')
    print(f'✅ {len(visible)} alvos visíveis\n')
    
    plt.close('all')
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
else:
    print(f'❌ Nenhum alvo visível às 22:00 UTC')

In [ ]:
# Meia-Noite (00:00 UTC)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

hora_escolhida = 0

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if hora_escolhida >= 12:
    map_dt = base_dt.replace(hour=hora_escolhida, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=hora_escolhida, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)
for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f'🗺️ Mapa do Céu - 00:00 UTC')
    print(f'✅ {len(visible)} alvos visíveis\n')
    
    plt.close('all')
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
else:
    print(f'❌ Nenhum alvo visível às 00:00 UTC')

In [ ]:
# Madrugada (02:00 UTC)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

hora_escolhida = 2

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if hora_escolhida >= 12:
    map_dt = base_dt.replace(hour=hora_escolhida, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=hora_escolhida, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)
for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f'🗺️ Mapa do Céu - 02:00 UTC')
    print(f'✅ {len(visible)} alvos visíveis\n')
    
    plt.close('all')
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
else:
    print(f'❌ Nenhum alvo visível às 02:00 UTC')

In [ ]:
# Fim da Noite (04:00 UTC)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

hora_escolhida = 4

# Calcular horário
base_dt = datetime.combine(DATA_ANALISE, datetime.min.time())
if hora_escolhida >= 12:
    map_dt = base_dt.replace(hour=hora_escolhida, minute=0)
else:
    map_dt = (base_dt + timedelta(days=1)).replace(hour=hora_escolhida, minute=0)

map_time = Time(map_dt)

# Filtrar alvos visíveis
visible = {}
frame = AltAz(obstime=map_time, location=observer_location)
for name, coord in all_targets.items():
    if coord:
        altaz = coord.transform_to(frame)
        if altaz.alt.deg >= ELEVACAO_MINIMA_GRAUS:
            visible[name] = coord

# Gerar mapa
if visible:
    print(f'🗺️ Mapa do Céu - 04:00 UTC')
    print(f'✅ {len(visible)} alvos visíveis\n')
    
    plt.close('all')
    fig = plot_sky_map(visible, observer_location, map_time)
    plt.show()
else:
    print(f'❌ Nenhum alvo visível às 04:00 UTC')

### 🎚️ Versão com Slider Interativo

Esta célula usa `interact_manual`:
- Mova o slider para escolher a hora
- Clique no botão "Run Interact"
- O mapa será gerado!

---

## 📅 4. Execução da Análise Anual

Esta seção gera um calendário de visibilidade para um alvo específico ao longo de um ano inteiro.

**Atenção:** Esta análise pode levar alguns minutos para ser concluída.

In [ ]:
if observer_location is not None and ALVO_ANUAL:
    # Buscar coordenadas do alvo
    target_coords = get_target_skycoords([ALVO_ANUAL])
    
    if ALVO_ANUAL in target_coords and target_coords[ALVO_ANUAL] is not None:
        print(f"🔭 Analisando visibilidade anual de {ALVO_ANUAL} para {ANO_ANALISE}...")
        
        # Análise anual
        df_year = analyze_year_visibility(
            ANO_ANALISE, observer_location, observer_timezone, 
            target_coords[ALVO_ANUAL], ELEVACAO_MINIMA_GRAUS * u.deg
        )
        
        if not df_year.empty:
            print(f"\n✅ {ALVO_ANUAL} foi visível em {len(df_year)} noites durante {ANO_ANALISE}!")
            
            # Gerar heatmap
            fig = plot_yearly_visibility(df_year, ALVO_ANUAL, ANO_ANALISE)
            if fig:
                plt.show()
            
            # Estatísticas
            print(f"\n📊 Estatísticas:")
            print(f"  - Noites visíveis: {len(df_year)}")
            print(f"  - Duração média: {df_year['duration_hours'].mean():.2f} horas")
            print(f"  - Duração máxima: {df_year['duration_hours'].max():.2f} horas")
        else:
            print(f"❌ {ALVO_ANUAL} não foi visível em nenhuma noite de {ANO_ANALISE}")
    else:
        print(f"❌ Não foi possível encontrar o alvo '{ALVO_ANUAL}'")
else:
    print("❌ Localização ou alvo anual não definidos")

---

## 🎓 Próximos Passos

- Experimente diferentes datas e localizações
- Adicione seus próprios alvos na lista `alvos_manuais`
- Ajuste a elevação mínima conforme suas condições de observação
- Explore o código-fonte em `src/` para entender os cálculos

**Boas observações! 🔭✨**